In [46]:
import json
from collections import defaultdict
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import random
import string
import scipy.optimize
import scipy.stats
import numpy as np
   
typeDict = defaultdict(int)#to store index for each wine type
index=0
data = json.load(open("winemag-data-130k-v2.json"))
review=[]
for l in data:
  review.append(l)
  if l['variety'] in typeDict: continue
  typeDict[l['variety']]=index
  index +=1
    
random.shuffle(review)

review_train = review[:int(len(review)/3)]
review_validation = review[int(len(review)/3):int(len(review)/3*2)]
review_test = review[int(len(review)/3*2):]
  

In [48]:
import string
from nltk.corpus import stopwords
stop_list = stopwords.words("english")
wordFreq = defaultdict(float)
wordFreqCat = {}
mydiff=defaultdict(list)
for c in range(len(typeDict)):
  wordFreqCat[c] = defaultdict(float)

punctuation = string.punctuation

for r in review_train:
  rText = ''.join([c for c in r['description'].lower() if not c in punctuation])
  for w in rText.split():
    if w in stop_list:continue
    wordFreq[w] += 1.0 # Word frequency
    wordFreqCat[typeDict[r['variety']]][w] += 1.0 # Per-category frequency

totalWords = sum(wordFreq.values())
for w in wordFreq:
  wordFreq[w] /= totalWords
for c in range(len(typeDict)):
  totalWordsCat = sum(wordFreqCat[c].values())
  for w in wordFreqCat[c]:
    wordFreqCat[c][w] /= totalWordsCat

for c in range(len(typeDict)):
  diffs = []
  for w in wordFreq:
    diffs.append((wordFreqCat[c][w] - wordFreq[w], w))
  diffs.sort()
  diffs.reverse()
  topten = diffs[:10]
  for r in topten: mydiff[c].append(r[1])
        
most_common = [(wordFreq[w], w) for w in wordFreq]
most_common.sort()
most_common.reverse()
most_common_words = [w[1] for w in most_common[:500]]
commonWords = most_common_words
word_set = set()
for w in commonWords:
  word_set.add(w)

pos = dict(zip(commonWords, range(len(commonWords))))



In [33]:
commonWords

['wine',
 'flavors',
 'fruit',
 'aromas',
 'palate',
 'finish',
 'acidity',
 'tannins',
 'drink',
 'cherry',
 'ripe',
 'black',
 'red',
 'notes',
 'spice',
 'rich',
 'fresh',
 'nose',
 'oak',
 'berry',
 'dry',
 'plum',
 'soft',
 'fruits',
 'blend',
 'crisp',
 'blackberry',
 'offers',
 'apple',
 'white',
 'sweet',
 'texture',
 'dark',
 'shows',
 'light',
 'citrus',
 'bright',
 'cabernet',
 'well',
 'vanilla',
 'full',
 'pepper',
 'juicy',
 'good',
 'fruity',
 'raspberry',
 'firm',
 'green',
 'peach',
 'touch',
 'lemon',
 'chocolate',
 'character',
 'years',
 'balanced',
 'sauvignon',
 'pear',
 'dried',
 'structure',
 'spicy',
 'smooth',
 'pinot',
 'made',
 'tannic',
 'herb',
 'like',
 'herbal',
 'concentrated',
 'also',
 'wood',
 'tart',
 'note',
 'bit',
 'mineral',
 'licorice',
 'merlot',
 'mouth',
 'flavor',
 'hint',
 'fine',
 'still',
 'give',
 'long',
 'toast',
 'balance',
 'creamy',
 'currant',
 'theres',
 'clean',
 'along',
 'age',
 'opens',
 'alongside',
 'orange',
 'structured',

In [54]:

def feature(r):
  feat = [0] * len(commonWords)
  raw = ''.join([x for x in r['description'].lower() if not x in string.punctuation])
  for word in raw.split():
    if word in word_set:
      feat[pos[word]] +=1
  
  
    
  return feat
    
x_train = [feature(r) for r in review_train]
y_train = [typeDict[r['variety']] for r in review_train]
           
x_validation = [feature(r) for r in review_validation]
y_validation = [typeDict[r['variety']] for r in review_validation]



In [55]:
from sklearn.naive_bayes import BernoulliNB
clf = MultinomialNB()
clf.fit(x_train, y_train)
predict = clf.predict(x_validation)
acc = [(x==y) for (x,y) in zip(predict, y_validation)] 
acc = sum(acc)*1.0/len(acc)  

print(acc)

0.426368756348


In [51]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train, y_train)
predict = clf.predict(x_validation)
acc = [(x==y) for (x,y) in zip(predict, y_validation)] 
acc = sum(acc)*1.0/len(acc)  

print(acc)


0.499099806112


In [ ]:
clf = OneVsRestClassifier(LinearSVC(C=0.1,random_state=0))
clf.fit(x_train, y_train)
predict = clf.predict(x_validation)
acc = [(x==y) for (x,y) in zip(predict, y_validation)] 
acc = sum(acc)*1.0/len(acc)  

print(acc)

In [42]:
for i in range(len(typeDict)): 
    sub_feat = [0] * 10
    common0= set(mydiff[i])
    pos0 = dict(zip(common0, range(10)))
    raw = ''.join([x for x in r['description'].lower() if not x in string.punctuation])
    for word in raw.split():
      if word in common0:
        sub_feat[pos0[word]] =1 
        
    feat+=sub_feat

[[0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
